# Imports

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
from google.colab import files
from tqdm.notebook import tqdm

#Pipeline de procesado

## Subir el csv

In [12]:
uploaded = files.upload()
for filename in uploaded.keys():
  dataframe = pd.read_csv(filename)
  dataframe.head()

Saving english_df_part_7.csv to english_df_part_7.csv


## Calcular sentiment análisis

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import pandas as pd
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

reviews = dataframe['review']
for index, text in tqdm(enumerate(reviews), total=len(reviews)):
  encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]
      dataframe.at[index, l] = s

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/2421 [00:00<?, ?it/s]

In [14]:
dataframe.head()

,id,review,type_review,date,language,views,votes,score,negative,neutral,positive
0,Porousword984,"Honestly, the first half of this game I love i...",normal,2020-06-22,English,4,3,1,0.895612,0.073294,0.031094
1,StableKable,Great looking game. Terrible SJW politics. Don...,expanded,2020-06-22,English,4,3,1,0.808430,0.133811,0.057760
2,dmh89,After waiting for so long for this game and th...,expanded,2020-06-22,English,4,3,1,0.915069,0.077935,0.006995
3,Koome,Graphics and animations are amazing but they d...,expanded,2020-06-22,English,4,3,1,0.876780,0.100232,0.022988
4,xNyaruko,Absolutely abysmal. Completely destroys the im...,expanded,2020-06-22,English,11,8,1,0.929779,0.059246,0.010976


## Descargar los resultados de salida

In [15]:
filename = f'sent_english_part_.csv'
dataframe.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>